In [ ]:
class HiddenNumber:
    
    def __init__(self,p):
        #gerar o segredo
        self.secret = ZZ.random_element(p)
        print 'The Secret is: '; print self.secret
    
    '''
    Aproximação dos k bits mais significativos do seu 
    argumento módulo p
    '''
    def msb(self,k,p,xi):
        val = ZZ(Mod(xi*self.secret,p))
        bits = val.digits(2)
        bits.reverse()
        final = ''
        if len(bits) < k:
            final = ''.join(str(e) for e in bits)
        else :
            for i in range(0,k):
                final += str(bits[i])
        return final

    def compare(self,calc):
        if calc == self.secret:
            return True
        else:
            return False
    


In [ ]:
# l inteiros positivos no intervalo {0,p-1}
def generate_xvector(l,p):
    x = []
    for i in range(0,l):
        x.append(ZZ.random_element(p))
    return x

'''
xv -> x vector
k -> número > 0
p -> primo

vetor de l inteiros positivos que verificarem as igualdades
ui = msb(s*xi)
'''
def generate_uvector(xv,k,p,hnp):
    u = []
    for xi in xv:
        ui = hnp.msb(k,p,xi)
        #binario para decimal
        ui = ZZ(int(ui,2))
        u.append(ui)
    return u
    


In [ ]:
import sage.modules.free_module_integer as fmi
import numpy as np

class BV:
    
    def __init__(self,uv,xv,p,k,l):
        
        #gerar lambda, matriz L, target T e reticulado
        self.comp_lambda = 2^(k+1)
        self.L = self.comp_lambda * p * matrix.identity(l)
        self.L = self.L.transpose()
        self.L = self.L.insert_row(l,zero_vector(l))
        self.L = self.L.transpose()
        x = [self.comp_lambda * i for i in xv]
        x.append(1)
        self.L = self.L.insert_row(l,x)
        
        #gerar matriz target
        self.target = [self.comp_lambda * i for i in uv]
        self.target.append(0)
        self.target = matrix(self.target)
        self.retic = fmi.IntegerLattice(self.L)
    
    def CVP_resolve(self,x,p,l):
        L = matrix(self.retic.reduced_basis)
        t = matrix(1,l+1,list(-self.target))
        zero = matrix(l+1,1,[0]*(l+1))
        M = matrix(1,1,[p**2])
        L1 = block_matrix(2,2,[[L,zero],[t,M]])
        #base reduzida do reticulado de base L1
        Lred = fmi.IntegerLattice(L1).reduced_basis
        err1 = np.array(Lred[l+1][:-1])
        y1 = err1 + self.target
        # última componente do vetor resultante
        return y1[0][l]
        

In [ ]:
l = 2^7
k = 64
p = 2^64
xv = generate_xvector(l,p)
hnp = HiddenNumber(p)
uv = generate_uvector(xv,k,p,hnp)
bv = BV(uv,xv,p,k,l)
calc = bv.CVP_resolve(xv,p,l)
if hnp.compare(calc):
    print 'True'
else:
    print 'False'